In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import QUANTAXIS as QA
import datetime

#设定绘图的默认大小
import matplotlib
matplotlib.rcParams["figure.figsize"]=[16,8]

#加载 seaborn，并且设置默认使用 seaborn
import seaborn as sns
sns.set()

matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus']=False #用来正常显示负号

import os

#数据所在目录
p=os.path.join(os.path.abspath(os.curdir),'data')

#日线数据所在目录
daily_path=os.path.join(p,'daily')

#日线数据开始日期
s='1990-01-01'
#日线数据结束日期
e='2018-12-31'

#使用的指数代码
zs_symbol='000300'

In [12]:
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Timestamp.html
QA.QA_fetch_index_day_adv(zs_symbol,s,e).index[0]

(Timestamp('2005-01-04 00:00:00'), '000300')

In [21]:
#IPO截止日期,000300的数据从这天以后才有，所以从这天开始开始，后面上市的暂不统计
ipo_date=QA.QA_fetch_index_day_adv(zs_symbol,s,e).index[0][0].date()
ipo_date

datetime.date(2005, 1, 4)

In [187]:
#指数日线数据
zs_daily=QA.QA_fetch_index_day_adv(zs_symbol,s,e)
zs_daily=zs_daily.to_pd().reset_index().set_index('date').drop(columns=['code'])
zs_daily.head()

,open,high,low,close,up_count,down_count,volume,amount
date,,,,,,,,
2005-01-04,994.76,994.76,980.65,982.79,0,0,74128.0,4.431976e+09
2005-01-05,981.57,997.32,979.87,992.56,0,0,71191.0,4.529207e+09
2005-01-06,993.33,993.78,980.33,983.17,0,0,62880.0,3.921015e+09
2005-01-07,983.04,995.71,979.81,983.95,0,0,72986.0,4.737468e+09
2005-01-10,983.76,993.95,979.78,993.87,0,0,57916.0,3.762931e+09


In [101]:
def calc_daily_return(l):
    """计算日收益"""
    return (l/l.shift(1)-1).dropna()

In [188]:
#指数日收益
zs_daily_return=calc_daily_return(zs_daily['close'])
zs_daily_return.head()

date
2005-01-05    0.009941
2005-01-06   -0.009460
2005-01-07    0.000793
2005-01-10    0.010082
2005-01-11    0.003280
Name: close, dtype: float64

In [191]:
#单只股票日线数据
stock_daily=QA.QA_fetch_stock_day_adv('600000',s,e)
stock_daily=stock_daily.to_pd().reset_index().set_index('date').drop(columns=['code'])
stock_daily.head()

,open,high,low,close,volume,amount
date,,,,,,
1999-11-10,29.50,29.80,27.00,27.75,1740849.0,4.859102e+09
1999-11-11,27.58,28.38,27.53,27.71,294034.0,8.215822e+08
1999-11-12,27.86,28.30,27.77,28.05,150079.0,4.215916e+08
1999-11-15,28.20,28.25,27.70,27.75,119210.0,3.329528e+08
1999-11-16,27.88,27.97,26.48,26.55,232231.0,6.289083e+08


In [192]:
#单只股票日收益
stock_daily_return=calc_daily_return(stock_daily['close'])
stock_daily_return.head()

date
1999-11-11   -0.001441
1999-11-12    0.012270
1999-11-15   -0.010695
1999-11-16   -0.043243
1999-11-17    0.023729
Name: close, dtype: float64

In [197]:
daily_return=pd.DataFrame({zs_symbol:zs_daily_return,'600000':stock_daily_return})
daily_return.dropna().head()

,000300,600000
date,,
2005-01-05,0.009941,-0.015988
2005-01-06,-0.009460,-0.013294
2005-01-07,0.000793,0.002994
2005-01-10,0.010082,0.034328
2005-01-11,0.003280,0.011544


In [206]:
np.polyfit(daily_return['600000'],daily_return['000300'],1)

LinAlgError: SVD did not converge in Linear Least Squares